In [3]:
import geopandas as gpd
import pandas as pd
import os, sys
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import LoadOSM as losm
import importlib
import networkx as nx
import osmnx as ox
from pulp import LpInteger,LpVariable, LpProblem, lpSum, LpMinimize
from shapely.geometry import LineString, MultiLineString, Point

In [4]:
#read back in processed graph
G = nx.read_gpickle('/home/vagrant/Downloads/lima_salvador/biggest_subg.pickle')

In [5]:
G.number_of_edges()

355958

In [7]:
#time was already added
gn.example_edge(G)

(0, 47401, {'Wkt': 'LINESTRING (-77.040189 -11.9019336, -77.0399958 -11.901326)', 'id': 142447, 'infra_type': 'primary_link', 'osm_id': '435838139', 'key': 'edge_142447', 'length': 70.43204956861656, 'Type': 'legitimate', 'time': 12.677768922350982, 'mode': 'drive'})


In [28]:
#make sure origin and destination are created. They will be CSVs with 'Lat' and 'Lon' columns.
#There will also need to be a way in the future to add a 'demand' column for the origins CSV, and a 'capacity' column for the destinations dataset.
origins_gdf = pd.read_csv('/home/vagrant/Downloads/lima_salvador/origins_pop_salvador_snapped2.csv')

In [29]:
origins_gdf

,field_1,ID,NC_CLASS,NC_COD00,NC_DES00,SHAPE_AREA,SHAPE_LEN,ORIG_FID,Lon,Lat,geometry,NN,NN_dist,total_dist,Population
0,1145,1162,3,1121,1.1.2.1. - Discontinuous dense urban fabric,30890.64680,877.72702,1119,-76.92767,-12.24319,POINT (-76.9276655802 -12.2431932923),97724,36.95009,431.775853,503
1,1146,1163,3,1121,1.1.2.1. - Discontinuous dense urban fabric,28998.64310,1379.87199,1120,-76.91654,-12.24424,POINT (-76.9165428507 -12.2442392571),89182,13.04458,438.050881,472
2,1147,1164,3,1121,1.1.2.1. - Discontinuous dense urban fabric,9523.82940,448.62038,1121,-76.93012,-12.24269,POINT (-76.93012424279999 -12.2426873632),13548,34.18857,495.427755,155
3,1148,1165,3,1121,1.1.2.1. - Discontinuous dense urban fabric,12832.26430,662.63793,1122,-76.92808,-12.24249,POINT (-76.9280755993 -12.2424942503),76580,28.34336,458.407808,209
4,1149,1166,3,1121,1.1.2.1. - Discontinuous dense urban fabric,58535.56390,1304.70139,1123,-76.91687,-12.24332,POINT (-76.91687197029999 -12.2433162641),86498,33.58559,408.265840,953
5,1150,1167,3,1121,1.1.2.1. - Discontinuous dense urban fabric,87123.83620,2263.26838,1124,-76.93117,-12.24378,POINT (-76.9311685516 -12.2437784917),43562,47.32597,414.916139,1419
6,1151,1168,3,1121,1.1.2.1. - Discontinuous dense urban fabric,6652.84730,383.66275,1125,-76.93114,-12.24186,POINT (-76.9311440426 -12.2418582138),109206,10.46430,500.668686,108
7,1152,1169,3,1121,1.1.2.1. - Discontinuous dense urban fabric,14402.92925,513.40299,1126,-76.93366,-12.24178,POINT (-76.9336557041 -12.2417759863),95661,6.24602,521.318037,235
8,1153,1170,3,1121,1.1.2.1. - Discontinuous dense urban fabric,11635.47870,446.15473,1127,-76.92857,-12.24168,POINT (-76.9285702435 -12.2416829233),47487,26.95888,514.594556,190
9,1154,1171,3,1121,1.1.2.1. - Discontinuous dense urban fabric,16556.45130,515.32908,1128,-76.91438,-12.24183,POINT (-76.914376855 -12.2418296019),105242,33.02702,452.633305,270


In [30]:
origins_w_demands = pd.Series(origins_gdf.Population.values,index=origins_gdf.NN).to_dict()

In [31]:
len(origins_w_demands)

620

In [32]:
origins_w_demands

{97724: 503,
 89182: 472,
 13548: 155,
 76580: 209,
 86498: 953,
 43562: 1419,
 109206: 108,
 95661: 235,
 47487: 190,
 105242: 270,
 82199: 272,
 54989: 172,
 54499: 135,
 108665: 482,
 105208: 155,
 19788: 52,
 35758: 1502,
 28502: 793,
 89844: 104,
 16814: 346,
 18559: 47,
 70021: 60,
 28990: 395,
 111827: 254,
 50147: 376,
 75824: 56,
 87466: 53,
 57045: 390,
 117091: 128,
 15057: 498,
 28948: 722,
 30093: 1290,
 101133: 268,
 39516: 134,
 21198: 749,
 29240: 413,
 83707: 54,
 77390: 125,
 3056: 429,
 68331: 857,
 29970: 459,
 766: 310,
 53757: 412,
 68036: 249,
 86438: 802,
 6997: 280,
 17806: 104,
 35920: 428,
 48971: 296,
 54554: 2403,
 103131: 125,
 45949: 643,
 63700: 151,
 108673: 1848,
 13800: 976,
 52824: 1184,
 8720: 257,
 16367: 762,
 43894: 560,
 61808: 90,
 53396: 853,
 52625: 244,
 82387: 261,
 106356: 841,
 57168: 578,
 14767: 781,
 98228: 120,
 107355: 179,
 111424: 61,
 33056: 562,
 6132: 328,
 33365: 120,
 17152: 152,
 36541: 53,
 23571: 416,
 75112: 814,
 36384: 1

In [38]:
destinations_gdf = pd.read_csv('/home/vagrant/Downloads/lima_salvador/Nodes_all_D_VilaElSalvador.csv')

In [39]:
destinations_gdf

,Field1,institucio,codigo,nombre,tipo,departamen,provincia,distrito,ubigeo,cat,categoria,Lat,Lon,ID,geometry,NN,NN_dist
0,337,MINSA,7278,PUESTO DE SALUD HEROES DEL CENEPA,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,12,-12.248749,-76.930702,337,POINT (-76.93070221000001 -12.24874878),26684,17.668383
1,338,MINSA,7716,SASBI,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,12,-12.208811,-76.955727,338,POINT (-76.95572661999999 -12.20881081),102330,16.737332
2,339,MINSA,6124,CESAR LOPEZ SILVA,Con Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,14,-12.230375,-76.923637,339,POINT (-76.92363739 -12.23037529),66722,46.775556
3,340,MINSA,6125,SAN MARTIN DE PORRES,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,13,-12.212154,-76.939438,340,POINT (-76.93943787000001 -12.21215439),98810,38.959049
4,341,MINSA,6126,PRINCIPE DE ASTURIAS,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,12,-12.230355,-76.911362,341,POINT (-76.91136169000001 -12.23035526),23688,20.278953
5,342,MINSA,6127,PUESTO DE SALUD PACHACAMAC,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,12,-12.224347,-76.922035,342,POINT (-76.92203522 -12.22434711),69282,56.753151
6,343,MINSA,6128,PUESTO DE SALUD EDILBERTO RAMOS,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,12,-12.209316,-76.935165,343,POINT (-76.93516541 -12.20931625),89362,27.084404
7,344,MINSA,6129,BRISAS DE PACHACAMAC,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,12,-12.201386,-76.948044,344,POINT (-76.94804382 -12.2013855),50659,23.288640
8,346,MINSA,6131,SAGRADA FAMILIA,Sin Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,12,-12.209131,-76.943787,346,POINT (-76.94378662 -12.20913124),44019,18.449342
9,347,MINSA,6132,CENTRO MATERNO INFANTIL SAN JOSE,Con Internamiento,LIMA,LIMA,VILLA EL SALVADOR,150142,1,14,-12.198921,-76.945900,347,POINT (-76.94589996000001 -12.1989212),22507,57.513439


In [40]:
destinations = list(destinations_gdf.NN)
destinations = list(set(destinations))

In [41]:
destinations

[62945,
 66722,
 69282,
 50659,
 57573,
 23688,
 22507,
 64877,
 101487,
 34128,
 98810,
 89362,
 44019,
 52951,
 9304,
 102330,
 26684]

In [42]:
%time OD_weighted_demand = gn.calculate_OD(G, origins_w_demands, destinations, fail_value = 9999999999999, weighted_origins=True)

print origins type
<class 'dict'>
weighted_origins equals true
CPU times: user 4min 34s, sys: 162 ms, total: 4min 34s
Wall time: 4min 38s


In [43]:
#should be the number of rows (which represent origins)
len(OD_weighted_demand)

620

In [44]:
OD_weighted_demand

array([[ 497722.14687218,  389873.2135446 ,  428001.99160979, ...,
         809440.68164361,  974532.67134673,  198597.3595611 ],
       [ 617471.95009986,  344800.09555872,  292037.97091605, ...,
         649968.51106707,  977901.55202341,  200602.97345712],
       [ 128588.93037103,  112516.56945888,  153229.40168545, ...,
         255548.22352599,  275518.61485126,   71492.07358273],
       ...,
       [1909415.0419345 , 1603707.43738806, 1321130.31819901, ...,
        1177739.44623002, 1010814.90393063, 2012578.02800567],
       [ 437979.77648529,  290481.94798089,  320249.54718169, ...,
         632680.44239826,  843403.73737922,  157760.94007402],
       [ 315943.82409227, 1043523.11301103, 1060719.36478806, ...,
         539491.81774463,  500375.5706669 , 1213159.66577501]])

In [45]:
OD_weighted_demand_df = pd.DataFrame(OD_weighted_demand, columns = destinations, index = origins_w_demands.keys())

In [46]:
OD_weighted_demand_df

,62945,66722,69282,50659,57573,23688,22507,64877,101487,34128,98810,89362,44019,52951,9304,102330,26684
97724,4.977221e+05,3.898732e+05,4.280020e+05,9.610045e+05,7.889175e+05,3.089627e+05,9.227014e+05,4.196560e+05,7.790739e+05,1.024893e+06,8.284200e+05,7.914967e+05,8.715630e+05,6.568750e+05,8.094407e+05,9.745327e+05,1.985974e+05
89182,6.174720e+05,3.448001e+05,2.920380e+05,7.921914e+05,7.907875e+05,1.803351e+05,7.562489e+05,2.842063e+05,6.214732e+05,8.946995e+05,7.203263e+05,6.331304e+05,7.245812e+05,7.736442e+05,6.499685e+05,9.779016e+05,2.006030e+05
13548,1.285889e+05,1.125166e+05,1.532294e+05,3.158162e+05,2.354825e+05,1.165473e+05,3.056716e+05,1.323218e+05,2.614126e+05,3.081986e+05,2.476552e+05,2.610777e+05,2.609498e+05,1.947935e+05,2.555482e+05,2.755186e+05,7.149207e+04
76580,1.994421e+05,1.546368e+05,1.852027e+05,4.066690e+05,3.204425e+05,1.357410e+05,3.907537e+05,1.813419e+05,3.310755e+05,4.184920e+05,3.368561e+05,3.362373e+05,3.547823e+05,2.655779e+05,3.436931e+05,3.975602e+05,8.988347e+04
86498,1.198757e+06,6.482246e+05,5.416943e+05,1.551538e+06,1.548703e+06,3.161586e+05,1.478968e+06,5.258817e+05,1.206846e+06,1.758509e+06,1.406437e+06,1.230383e+06,1.415028e+06,1.514079e+06,1.264380e+06,1.926499e+06,3.570703e+05
43562,1.224869e+06,1.154972e+06,1.269903e+06,2.773542e+06,2.277042e+06,9.340839e+05,2.665486e+06,1.246358e+06,2.260302e+06,2.942747e+06,2.388482e+06,2.295348e+06,2.510191e+06,1.903838e+06,2.345969e+06,2.569986e+06,5.216105e+05
109206,8.435838e+04,7.950864e+04,1.098945e+05,2.165611e+05,1.605866e+05,8.433540e+04,2.161123e+05,9.340398e+04,1.852738e+05,2.112534e+05,1.690683e+05,1.830222e+05,1.783316e+05,1.321821e+05,1.791694e+05,1.867352e+05,5.294207e+04
95661,1.681917e+05,2.037485e+05,2.449663e+05,4.642379e+05,3.424417e+05,1.893515e+05,4.760882e+05,2.339837e+05,4.089859e+05,4.526888e+05,3.608972e+05,3.994663e+05,3.810535e+05,2.806356e+05,3.910829e+05,3.909561e+05,1.210419e+05
47487,1.721853e+05,1.297169e+05,1.793046e+05,3.789228e+05,2.804493e+05,1.343395e+05,3.661692e+05,1.539943e+05,3.119163e+05,3.695852e+05,2.953707e+05,3.118241e+05,3.116673e+05,2.305724e+05,3.050460e+05,3.541222e+05,9.265078e+04
105242,3.517569e+05,1.770073e+05,1.409292e+05,4.270340e+05,4.262309e+05,7.703140e+04,4.064736e+05,1.364493e+05,3.293774e+05,4.856721e+05,3.859247e+05,3.360457e+05,3.883586e+05,4.408623e+05,3.456776e+05,5.332665e+05,1.132937e+05


In [47]:
facilities = OD_weighted_demand_df.columns.values.tolist()

In [51]:
result_w_demands = gn.optimize_facility_locations(OD_weighted_demand_df, facilities, 10, existing_facilities = None)

In [52]:
result_w_demands

[101487, 102330, 22507, 23688, 26684, 34128, 52951, 62945, 66722, 98810]